# Demonstrateur pour la plateforme web-cartographique d'indicateurs  - 
# #1. découpages géographiques pour visualiser un indicateur sur une zone d'intérêt

Ce notebook a pour objectif de proposer un premier aperçu visuel de certains indicateurs à travers des cartes interactives.   
Ces visualisations serviront de base de discussion pour la conception de la plateforme web dédiée.  

Les données sont extraites depuis notre base de données **PostgreSQL/PostGIS**.

🛠️ Les principaux outils utilisés sont :
- **Pandas** : pour le traitement et la manipulation des données à partir de la BDD.
- **Folium** : pour la création de cartes interactives en Python.

📋 On présente ici les **différents découpages géographiques accessibles à l'utilisateur de la plateforme** (liste non exhaustive) pour projeter les calculs des indicateurs. On souhaite que l'utilisateur puisse à accéder à tous ces découpages pour chacun des indicateurs sélectionnés.  
La zone d'intérêt est ici une Aire d'Alimentation de Captage (AAC) de Cerou Vere.  
La liste des découpages géographiques présentés : 
- régions hydrographiques
- secteurs hydrographiques
- bassins versants
- régions
- départements
- PRA
- SCOT
- EPCI
- communes
- carreaux 10km

Ce notebook est un prototype visant à tester la lisibilité et la pertinence des représentations cartographiques.  

In [ ]:
import psycopg2
import geopandas as gpd
import json
# pour la visualisation :
from matplotlib import colormaps
import folium
from folium.plugins import FloatImage
from folium.plugins import GroupedLayerControl
from folium.plugins import MiniMap, Draw

## Connexion à la BDD

In [ ]:
def connection_2_bdd(uname, password, host, port, database_name):
    """Connection via psycopg2 à la BDD PostgreSQL"""
    return psycopg2.connect(
        user=uname, password=password, host=host, port=port, database=database_name
    )

In [ ]:
# Lecture des params de connexion dans un json dédié
fileObject = open("../param_connexion_BDD.json", mode="r", encoding="utf-8")
jsonContent = fileObject.read()
param_connexion = json.loads(jsonContent)

conn = connection_2_bdd(
    param_connexion["username"],
    param_connexion["password"],
    param_connexion["host"],
    param_connexion["port"],
    param_connexion["bdd_name"],
)
curs = conn.cursor()

## Territoire d'intérêt

In [ ]:
# Requête SQL pour récupérer toutes les données
sql = f'SELECT * FROM app_webae_dev.territoire_test_bv_cerou_vere;'

# Lecture dans un GeoDataFrame via geopandas
gdf_aac = gpd.read_postgis(sql, conn, geom_col="geom")

# Affichage des premières lignes
gdf_aac = gdf_aac.to_crs(epsg=4326)
gdf_aac = gdf_aac[['geom']]

## Communes

In [ ]:
# Requête SQL pour récupérer toutes les communes sur le territoire
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
communes_territoires AS
(SELECT DISTINCT 'aac_coulonge_st_hyppolyte' AS territoire
,b.insee_com,nom_commune,insee_dep,nom_departement,insee_reg,nom_region,siren_epci,nom_epci,b.geom
      FROM territoire_limite a
      JOIN app_webae_dev.v_communes2025_fre_4326 b
      ON st_intersects(st_transform(a.geom,4326),st_transform(b.geom,4326))
)
SELECT DISTINCT a.geom,a.nom_commune, b.*
FROM communes_territoires a
LEFT JOIN app_webae_dev.mv_indicateurs2022_com2025_fr_rpg_seul b
ON a.insee_com=b.insee_com;
"""

gdf_communes = gpd.read_postgis(sql, conn, geom_col="geom")

## Carreaux de 10km

In [ ]:
# Requête SQL pour récupérer toutes les communes sur le territoire
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
carreaux_territoire AS (
    SELECT DISTINCT c.id_carreau, c.geom
    FROM app_webae_dev.ign_carreaux_10km_fr_2154 c
    JOIN territoire_limite t
    ON ST_Intersects(c.geom, t.geom)
)
SELECT c.geom, i.*
FROM carreaux_territoire c
JOIN app_webae_dev.mv_indicateurs2022_carreau10km_fr_rpg_seul i
ON c.id_carreau = i.id_carreau;
"""

gdf_carreaux = gpd.read_postgis(sql, conn, geom_col="geom")

## Départements

In [ ]:
# Requête SQL pour récupérer toutes les départements sur le territoire 
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
dpt_territoire AS (
	SELECT DISTINCT d.insee_dep, d.geom, d.nom
	FROM ref_administratif_2025.ign_admin_dep_fr_2154 d
	JOIN territoire_limite t
	ON ST_Intersects(d.geom, t.geom)
)
SELECT d.geom, d.nom, i.*
FROM dpt_territoire d
JOIN app_webae_dev.mv_indicateurs2022_dep2025_fr_rpg_seul i
ON d.insee_dep = i.insee_dep;
"""

gdf_dpt = gpd.read_postgis(sql, conn, geom_col="geom")

## Petites Régions Agricoles (PRA)

In [ ]:
# Requête SQL pour récupérer toutes les PRA sur le territoire 
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
pra_territoire AS (
	SELECT DISTINCT d.code_pra, d.geom, d.libelle_pra
	FROM ref_agricole.ofb_petites_regions_agricoles_fr_2154 d
	JOIN territoire_limite t
	ON ST_Intersects(d.geom, t.geom)
)
SELECT p.geom, p.libelle_pra, i.*
FROM pra_territoire p
JOIN app_webae_dev.mv_indicateurs2022_pra_fr i
ON p.code_pra = i.code_pra;
"""

gdf_pra = gpd.read_postgis(sql, conn, geom_col="geom")

## EPCI

In [ ]:
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
epci_territoire AS (
	SELECT DISTINCT e.code_siren, e.geom, e.nom
	FROM ref_administratif_2025.ign_admin_epci_fr_2154 e
	JOIN territoire_limite t
	ON ST_Intersects(e.geom, t.geom)
)
SELECT t.geom, t.nom, i.*
FROM epci_territoire t
JOIN app_webae_dev.mv_indicateurs2022_epci2025_fr_rpg_seul i
ON t.code_siren = i.siren_epci;
"""

gdf_epci = gpd.read_postgis(sql, conn, geom_col="geom")

## Régions

In [ ]:
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
reg_territoire AS (
	SELECT DISTINCT r.insee_reg, r.geom, r.nom
	FROM ref_administratif_2025.ign_admin_region_fr_2154 r
	JOIN territoire_limite t
	ON ST_Intersects(r.geom, t.geom)
)
SELECT t.geom, t.nom, i.*
FROM reg_territoire t
JOIN app_webae_dev.mv_indicateurs2022_reg2025_fr_rpg_seul i
ON t.insee_reg = i.insee_reg;
"""

gdf_reg = gpd.read_postgis(sql, conn, geom_col="geom")

## Bassin Versant

In [ ]:
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
bv_territoire AS (
	SELECT DISTINCT r.id_bassin_versant, r.geom, r.toponyme
	FROM ref_hydro.ign_topo_bassin_versant_topographique_fr_2154 r
	JOIN territoire_limite t
	ON ST_Intersects(r.geom, t.geom)
)
SELECT t.geom, t.toponyme, i.*
FROM bv_territoire t
JOIN app_webae_dev.mv_indicateurs2022_bassins_versants_fr_rpg_seul i
ON t.id_bassin_versant = i.id_bassin_versant;
"""

gdf_bv = gpd.read_postgis(sql, conn, geom_col="geom")

## Région Hydrographique

In [ ]:
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
rh_territoire AS (
	SELECT DISTINCT r.code_region_hy, r.geom, r.libelle_region_hy
	FROM ref_hydro.sandre_regions_hydrographiques_fr_2154 r
	JOIN territoire_limite t
	ON ST_Intersects(r.geom, t.geom)
)
SELECT t.geom, t.libelle_region_hy, i.*
FROM rh_territoire t
JOIN app_webae_dev.mv_indicateurs2022_rh2025_fr i
ON t.code_region_hy = i.code_region_hy;
"""

gdf_regHydro = gpd.read_postgis(sql, conn, geom_col="geom")

## Secteurs Hydrographique

In [ ]:
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
sh_territoire AS (
	SELECT DISTINCT r.code_secteur_hy, r.geom, r.libelle_secteur_hy
	FROM ref_hydro.sandre_secteurs_hydrographiques_fr_2154 r
	JOIN territoire_limite t
	ON ST_Intersects(r.geom, t.geom)
)
SELECT t.geom, t.libelle_secteur_hy, i.*
FROM sh_territoire t
JOIN app_webae_dev.mv_indicateurs2022_sh2025_fr i
ON t.code_secteur_hy = i.code_secteur_hy;
"""

gdf_secHydro = gpd.read_postgis(sql, conn, geom_col="geom")

## SCOT

In [ ]:
sql = """
WITH territoire_limite AS (
    SELECT * 
    FROM app_webae_dev.territoire_test_bv_cerou_vere
),
scot_territoire AS (
    SELECT DISTINCT c.title, c.fid, c.geom
    FROM app_webae_dev.ign_schemas_coherence_territoriale_fr_2154 c
    JOIN territoire_limite t
    ON ST_Intersects(c.geom, t.geom)
)
SELECT c.title, c.geom, i.*
FROM scot_territoire c
JOIN app_webae_dev.mv_indicateurs2022_scot_fr i
ON c.fid = i.fid_scot;
"""

gdf_scot = gpd.read_postgis(sql, conn, geom_col="geom")

## Visualisation des zones géographiques d'intérêt

In [ ]:
# On force le CRS
gdf_regHydro = gdf_regHydro.set_geometry('geom').to_crs(epsg=4326)
gdf_secHydro = gdf_secHydro.set_geometry('geom').to_crs(epsg=4326)
gdf_epci = gdf_epci.set_geometry('geom').to_crs(epsg=4326)
gdf_reg = gdf_reg.set_geometry('geom').to_crs(epsg=4326)
gdf_bv = gdf_bv.set_geometry('geom').to_crs(epsg=4326)
gdf_regHydro = gdf_regHydro.set_geometry('geom').to_crs(epsg=4326)
gdf_pra = gdf_pra.set_geometry('geom').to_crs(epsg=4326)
gdf_dpt = gdf_dpt.set_geometry('geom').to_crs(epsg=4326)
gdf_communes = gdf_communes.set_geometry('geom').to_crs(epsg=4326)
gdf_scot = gdf_scot.set_geometry('geom').to_crs(epsg=4326)
gdf_carreaux = gdf_carreaux.set_geometry('geom').to_crs(epsg=4326)
gdf_aac = gdf_aac.to_crs(epsg=4326)

# Centrage de la carte
center = gdf_aac.geometry.union_all().centroid
map_center = [center.y, center.x]

# Création de la carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=map_center, zoom_start=9, tiles=None).add_to(f)

# Liste de fonds de carte : (nom lisible, url, attribution)
tile_layers = [
    ('OpenStreetMap', 'OpenStreetMap', None),
    ('CartoDB Positron', 'CartoDB positron', None),
    ('Esri Satellite', 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 'Esri World Imagery'),
]

# Ajouter chaque fond à la carte
for name, tile_url, attr in tile_layers:
    if tile_url in ['OpenStreetMap', 'CartoDB positron']:
        folium.TileLayer(tile_url, name=name).add_to(m)
    else:
        folium.TileLayer(tiles=tile_url, attr=attr, name=name).add_to(m)

# Gestion du GroupedLayerControl 
fg1 = folium.FeatureGroup(name='régions hydrographiques', show=False)
fg2 = folium.FeatureGroup(name='secteurs hydrographiques', show=False)
fg4 = folium.FeatureGroup(name='bassins versants', show=False)
fg5 = folium.FeatureGroup(name='régions', show=False)
fg6 = folium.FeatureGroup(name='départements', show=False)
fg7 = folium.FeatureGroup(name='PRA', show=False)
fg8 = folium.FeatureGroup(name='SCOT', show=False)
fg9 = folium.FeatureGroup(name='EPCI', show=False)
fg10 = folium.FeatureGroup(name='communes', show=False)
fg11 = folium.FeatureGroup(name='carreaux 10km', show=False)

# Ajout de la couche AAC
folium.GeoJson(
    data=gdf_aac.to_json(),
    name="CEROU VERE",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 3, 'fillOpacity': 0.7},
).add_to(m)

# Ajoute des régions hydrographiques
folium.GeoJson(
    show=False,
    data=gdf_regHydro.to_json(),
    name="Régions hydrographiques",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['libelle_region_hy'],sticky=True, aliases=['Région hydro'])
).add_to(fg1)

# Ajoute des secteurs hydrographiques
folium.GeoJson(
    show=False,
    data=gdf_secHydro.to_json(),
    name="Secteurs hydrographiques",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['libelle_secteur_hy'],sticky=True, aliases=['Secteur hydro'])
).add_to(fg2)

# Ajout des bassins versants
folium.GeoJson(
    show=False,
    data=gdf_bv.to_json(),
    name="Bassins versants",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['toponyme'],sticky=True, aliases=['Bassin versant'])
).add_to(fg4)

# Ajout des régions
folium.GeoJson(
    show=False,
    data=gdf_reg.to_json(),
    name="Régions",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['nom'],sticky=True, aliases=['Région'])
).add_to(fg5)

# Ajout des départements
folium.GeoJson(
    show=False,
    data=gdf_dpt.to_json(),
    name="Départements",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['nom'],sticky=True, aliases=['Département'])
).add_to(fg6)

# Ajout des PRA
folium.GeoJson(
    show=False,
    data=gdf_pra.to_json(),
    name="PRA",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['libelle_pra'],sticky=True, aliases=['PRA'])
).add_to(fg7)

# Ajout des SCOT
folium.GeoJson(
    show=False,
    data=gdf_scot.to_json(),
    name="SCOT",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['title'],sticky=True, aliases=['SCOT'])
).add_to(fg8)

# Ajout des EPCI
folium.GeoJson(
    show=False,
    data=gdf_epci.to_json(),
    name="EPCI",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['nom'],sticky=True, aliases=['EPCI'])
).add_to(fg9)

# Ajout des communes
folium.GeoJson(
    show=False,
    data=gdf_communes.to_json(),
    name="Communes",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['nom_commune'],sticky=True, aliases=['Commune'])
).add_to(fg10)

# Ajout des carreaux de 10km
folium.GeoJson(
    show=False,
    data=gdf_carreaux.to_json(),
    name="Carreaux 10km",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['id_carreau'],sticky=True, aliases=['Carreau'])
).add_to(fg11)

# Ajout des couches à la carte
m.add_child(fg1)
m.add_child(fg2)
m.add_child(fg4)
m.add_child(fg5)
m.add_child(fg6)
m.add_child(fg7)
m.add_child(fg8)
m.add_child(fg9)
m.add_child(fg10)
m.add_child(fg11)
folium.LayerControl(collapsed=False).add_to(m)

# Regroupement des couches suivant différentes échelles spatiales
GroupedLayerControl(
    groups={'Unités hydrographiques': [fg1, fg2, fg4],
            'Territoires administratifs': [fg5, fg6, fg7, fg8, fg9, fg10],
            'Carreaux 10km IGN': [fg11]},
    collapsed=False,
    exclusive_groups=False,
).add_to(m)

# Ajout du logo Solagro
url_logo = ("https://osez-agroecologie.org/images/env/logo_solagro_hd.png")
FloatImage(url_logo, bottom=0, left=0, width='150px').add_to(m)

# Ajout de la minimap
MiniMap().add_to(m)

# Sauvegarde de la carte
m.save("carte_ZonesGeo_CerouVere.html")